In [1]:
import plotly.express as px
from rindti.models import NoisyNodesClassModel, ClassificationModel, InfoGraphModel, GraphLogModel, PfamModel, BGRLModel
from rindti.models.bgrl import Encoder
from rindti.utils.data import Dataset
import pickle
import pandas as pd
import seaborn as sns
from sklearn.manifold import TSNE

RDKit WARNING: [12:40:22] Enabling RDKit 2019.09.3 jupyter extensions


In [39]:
prots = pd.read_pickle("results/prepare_proteins/protein_data_whole_label_none.pkl")

In [41]:
prots['nnodes'] = prots['data'].apply(lambda x:x['x'].size(0))

In [43]:
prots[prots['nnodes'] < 1000][['data']].to_pickle("/scratch/SCRATCH_NVME/ilya/pretrain_data/prot_data_1000.pkl")

In [32]:
class TestModel(ClassificationModel):
    def embed_prot(self, x, edge_index, **kwargs):
        x = self.prot_feat_embed(x)
        x = self.prot_node_embed(x, edge_index)
        return self.prot_pool(x, edge_index, batch=None).detach().numpy().reshape(-1)
    
    def embed_drug(self, x, edge_index, **kwargs):
        x = self.drug_feat_embed(x)
        x = self.drug_node_embed(x, edge_index)
        return self.drug_pool(x, edge_index, batch=None).detach().numpy().reshape(-1)

In [33]:
def embed_prot(encoder, x, edge_index, **kwargs):
    embed, _ = encoder(x, edge_index, batch=None)
    return embed.detach().numpy().reshape(-1)

In [34]:

model = BGRLModel.load_from_checkpoint('tb_logs/bgrl/version_2/checkpoints/epoch=4-step=45649.ckpt')
model.eval()
encoder = model.student_encoder

In [35]:
embeds = {}
for id, prot in prots_sub['data'].to_dict().items():
    emb = embed_prot(encoder, **prot)
    embeds[id] = emb
prot_embeds = pd.DataFrame(embeds).T

In [36]:
tsne = TSNE()
x = tsne.fit_transform(prot_embeds)


In [37]:
x = pd.DataFrame(index=prots_sub.index, data=x).join(prots_sub)

In [38]:
px.scatter(x, 0, 1, hover_name=x.index, hover_data=['fam'], color='fam', opacity=0.5, size_max = 0.5)

In [ ]:
sns.clustermap(prot_embeds)

In [ ]:
help(sns.clustermap)